In [0]:
# Databricks notebook cell: 01_install_azure_eventhub

%pip install azure-eventhub

# Restart Python so the newly installed package is available in this notebook
dbutils.notebook.exit('Please run this notebook as the correct user or on the correct cluster.')



In [0]:
# Option A
secret_value = dbutils.secrets.get(
    scope="dbx-ss-kv-natraining-2", key="evh-natraining-read-write"
)
send_conn_str = (
    "Endpoint=sb://evhns-natraining.servicebus.windows.net/;"
    "SharedAccessKeyName=SharedAccessKeyToSendAndListen;"
    f"SharedAccessKey={secret_value};"
    "EntityPath=evh-natraining-001"
)
eh_namespace = "evhns-natraining.servicebus.windows.net"
eh_name = "evh-natraining-001"
dbutils.widgets.text("eventhub_name", "evh-natraining-001")
dbutils.widgets.text(
    "test_message", "Test Biju on Databricks!",
)

EVENTHUB_CONN_STR = send_conn_str
EVENTHUB_NAME = dbutils.widgets.get("eventhub_name")
TEST_MESSAGE = dbutils.widgets.get("test_message")

if not EVENTHUB_CONN_STR or not EVENTHUB_NAME:
    raise ValueError(
        "Please provide both 'eventhub_connection_string' and 'eventhub_name' via widgets."
    )

print("Parameters loaded. Ready to send a test message.")

# -------------------------------------------------------------------
# Option B (prod-style): use Databricks secrets instead of widgets
#
#
# EVENTHUB_CONN_STR = dbutils.secrets.get("azure-kv-scope", "eventhub-connstr")
# EVENTHUB_NAME = "my-eventhub"
# TEST_MESSAGE = "Hello from Databricks via secret!"
# -------------------------------------------------------------------

In [0]:
from azure.eventhub import EventHubProducerClient, EventData
import json
from datetime import datetime, timezone


def send_test_event(message: str) -> None:
    """Send a single test event and raise a clear error if anything fails."""
    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENTHUB_CONN_STR,
        eventhub_name=EVENTHUB_NAME,
    )

    payload = {
        "message": message,
        "source": "databricks-notebook-smoketest",
        "timestamp_utc": datetime.now(timezone.utc).isoformat(timespec="milliseconds"),
    }

    body = json.dumps(payload)

    with producer:
        batch = producer.create_batch()
        batch.add(EventData(body))
        producer.send_batch(batch)

    print(f"✅ Sent test event: {body}")


# Example usage for the course:
send_test_event(TEST_MESSAGE)